In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# III/ Comparaison de l'erreur de prédiction de différentes méthodes statistiques

Jusqu'à maintenant, nous avons démontré que les stratégies d'investissements obtenues à partir de la résolution du problème de Markovitz (équivalent à la résolution d'un problème des moindres carrés => démonstration dans le fichier LaTeX) ne sont pas très convaincantes. Nous allons donc tester plusieurs autres méthodes statistiques de prédiction des rendements futurs et allons mettre en place une métrique commune qui mesure l'erreur de prédiction obtenue. 



Ici on crée l'interface qui nous permet de choisir le nombre d'actifs considéré, le nombre de dates, la vraie loi des rendements

In [9]:
nb_actifs = 20
nb_dates = 256 * 5

idx = pd.Index(np.arange(nb_actifs))

esp_rdt = 0.05
vol_rdt = 0.2
correl = 0.7
mu = pd.Series(esp_rdt,index=idx)
vols = pd.Series(vol_rdt,index=idx)

#Matrice var-covar 
covar = np.diag(vols) @ pd.DataFrame(correl * np.ones((nb_actifs,nb_actifs)) + (1-correl) * np.eye(nb_actifs),index=idx,columns=idx) @ np.diag(vols)
A = np.linalg.cholesky(covar)

rdts_observes = mu/256 + pd.DataFrame(np.random.randn(nb_dates,nb_actifs)) @ A.T / 16

In [10]:
print(vols)

0     0.2
1     0.2
2     0.2
3     0.2
4     0.2
5     0.2
6     0.2
7     0.2
8     0.2
9     0.2
10    0.2
11    0.2
12    0.2
13    0.2
14    0.2
15    0.2
16    0.2
17    0.2
18    0.2
19    0.2
dtype: float64
